In [55]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle

In [56]:
# load saved model
with open('model_pkl' , 'rb') as f:
    model = pickle.load(f)

In [58]:
year = str(2018)
feature_df = pd.read_csv('feature_df'+year+'.csv')
df = feature_df[['away_past_average_yards/play',
'home_past_average_yards/play',
'away_past_starting_yard_line',
'home_past_starting_yard_line',
'away_schedule_strength',
'home_schedule_strength',
'spread', 
'margin',
'id']]


df.dropna(axis = 0, how = 'any', inplace = True)
df

labels = df.pop('margin')
#labels = labels.pop('margin_transform')

s_1 = df.pop('away_past_average_yards/play')
s_2 = df.pop('home_past_average_yards/play')
s_3 = df.pop('away_past_starting_yard_line')
s_4 = df.pop('home_past_starting_yard_line')
s_5 = df.pop('away_schedule_strength')
s_6 = df.pop('home_schedule_strength')
s_9 = df.pop('spread')*-1
s_10 = df.pop('id')

feature_df = pd.DataFrame()
feature_df = pd.concat([s_1, s_2, s_3, s_4, s_5, s_6,s_9,s_10, labels], axis=1).reset_index()
feature_df = feature_df.drop( 'index', axis = 1)
feature_df = np.array(feature_df)

#s_3 = df.pop("home_points")
#s_4 = df.pop("away_points")


#margin = df.pop('margin')
#validate_df = pd.concat([s_3,s_4,margin],axis = 1)
#validate_df = np.array(validate_df)


x_train = feature_df[:,[0,1,2,3,4,5,6]]
y_train = feature_df[:,8]
train_spread = feature_df[:,[6]]
train_id = feature_df[:,7]

#x_test = test[:,[0,1,2,3,4,5,6]]
#y_test = test[:,8]
#test_spread = test[:,[6]]
#test_id = test[:,7]

y_train = y_train*.01
#y_test = y_test*.01
print(np.shape(feature_df))
np_shape = np.shape(feature_df)

(645, 9)


/var/folders/70/ysch_yh10pz6qzjj3fsh3lfc0000gn/T/ipykernel_96894/3214772574.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis = 0, how = 'any', inplace = True)


In [59]:
#Evaluate for profitability
def get_profit(profit_arr):
    profit = 0
    for i in range(len(profit_arr)):
        game = profit_arr[i]
        if (game[1] < game[2]):
            if game[0] < game[2]:
                profit = profit + .9
            else:
                profit = profit - 1
        else:
            if game[0] > game[2]:
                profit = profit + .9
            else:
                profit = profit - 1
        
    
   
    return(profit)

def get_profit_2(profit_arr,br):
    profit = br
    
    for i in range(len(profit_arr)):
        unit = profit*.05
        game = profit_arr[i]
        if (game[1] < game[2]):
            if game[0] < game[2]:
                profit = profit + (unit*.9)
                
            else:
                profit = profit - unit
        else:
            if game[0] > game[2]:
                profit = profit + (unit*.9)
            else:
                profit = profit - unit
        
    
   
    return(print(profit-br))

In [61]:

prediction = model.predict(x_train)
spread = train_spread
spread = np.reshape(spread, (np_shape[0],1))
spread = spread*.01
print()
z = np.array([(y_train)])
z = z.swapaxes(0,1)

actual_scores = z
actual_vs_pred = np.concatenate((z, prediction), axis = 1)
profit_arr = np.concatenate((actual_vs_pred, spread), axis = 1)
correct_win = (np.sign(actual_scores) == np.sign(prediction))
difference = actual_scores-prediction
print("% of correct winning team")
print(correct_win.sum()/correct_win.size)
win_percent = correct_win.sum()/correct_win.size
print()
print("Mean")
mean = difference.mean()
print(difference.mean())
print()
print("Standard Deviation")
print(difference.std())
sd = difference.std()
print("")
print("Total Profit")
profit = get_profit(profit_arr)
print(profit)


21/21 [==============================] - 0s 3ms/step

% of correct winning team
0.751937984496124

Mean
-0.009793808389195175

Standard Deviation
0.15796989566356798

Total Profit
-75.00000000000023


In [38]:
export = pd.DataFrame(profit_arr)
export.to_csv('/Users/davidchristensen/Documents/Data Projects/Python/Sport_Analysis/double_check'+year+'.csv')